In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
import itertools as it

from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AR

from sklearn.model_selection import TimeSeriesSplit
from sklearn import metrics

import math

In [10]:
df = pd.read_csv('fitbit/tidy.csv')

In [11]:
df

,date,calories_burned,steps,distance,floors,minutes_sedentary,minutes_lightly_active,minutes_fairly_active,minutes_very_active,activity_calories
0,4/26/18,"2,635","2,931",1.38,2,"1,347",93,0,0,566
1,4/27/18,"3,649","9,971",4.70,19,"1,158",219,46,17,"1,752"
2,4/28/18,"3,379","3,521",1.66,2,702,155,22,11,"1,124"
3,4/29/18,"2,851","1,639",0.77,0,891,122,0,0,602
4,4/30/18,"2,825","1,323",0.62,0,880,104,0,0,540
5,5/1/18,"3,292","2,581",1.22,0,919,167,13,2,"1,068"
6,5/2/18,"2,660","1,372",0.65,0,927,90,0,0,419
7,5/3/18,"3,121","2,232",1.05,0,876,157,16,0,"1,006"
8,5/4/18,"2,522",904,0.43,1,"1,012",61,0,0,269
9,5/5/18,"2,551",833,0.39,0,969,68,0,0,326
